In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from scipy.integrate import solve_ivp
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm

In [ ]:
# Constants
G = 9.81  # Gravity (m/s^2)
BOUNCE_COEFF = 0.9  # Coefficient of restitution (energy retention on bounce)
dt = 0.01
mass = 1.0
radius = 0.2

In [ ]:
# Parameters
n_balls = 100
seq_len = 80
pred_len = 20
box_size = 5.0

In [ ]:
# Hyperparameters
latent_dim = 16
hidden_dim = 64
sparse_dim = 8
fourier_modes = 16
num_epochs = 100
batch_size = 16
learning_rate = 1e-3
alpha, beta, gamma = 0.1, 0.1, 0.1

In [ ]:
# Collision Handling
def handle_collisions(state, radius, box_size, bounce_coeff):
    x, vx, y, vy = state
    if x - radius < -box_size:
        x = -box_size + radius
        vx = -bounce_coeff * vx
    elif x + radius > box_size:
        x = box_size - radius
        vx = -bounce_coeff * vx
    if y - radius < -box_size:
        y = -box_size + radius
        vy = -bounce_coeff * vy
    elif y + radius > box_size:
        y = box_size - radius
        vy = -bounce_coeff * vy
    return np.array([x, vx, y, vy])

In [ ]:
# Particle Dynamics
def particle_dynamics_with_collisions(t, y, mass, radius, box_size, drag_coeff, bounce_coeff):
    x, vx, y, vy = y
    dydt = np.zeros_like(y)
    dydt[0] = vx
    dydt[2] = vy
    speed = np.sqrt(vx**2 + vy**2)
    drag_force_x = -drag_coeff * speed * vx
    drag_force_y = -drag_coeff * speed * vy
    dydt[1] = drag_force_x / mass
    dydt[3] = (-G + drag_force_y / mass)
    return dydt

In [ ]:
# Dataset Generation
def generate_ball_dataset(n_balls, dt, seq_len, pred_len, box_size=5.0):
    trajectories = []
    for _ in tqdm(range(n_balls), desc="Generating dataset"):
        mass = np.random.uniform(0.5, 5.0)
        radius = np.random.uniform(0.05, 0.2)
        drag_coeff = 0.1
        bounce_coeff = 0.9
        init_pos = np.random.uniform(-box_size + radius, box_size - radius, 2)
        init_vel = np.random.uniform(-5.0, 5.0, 2)
        y0 = np.concatenate([init_pos, init_vel])
        t_eval = np.linspace(0, (seq_len + pred_len) * dt, seq_len + pred_len)
        trajectory = []
        state = y0
        for t in t_eval:
            sol = solve_ivp(
                particle_dynamics_with_collisions,
                (t, t + dt),
                state,
                args=(mass, radius, box_size, drag_coeff, bounce_coeff),
                method="RK45",
                t_eval=[t + dt],
            )
            state = sol.y.flatten()
            state = handle_collisions(state, radius, box_size, bounce_coeff)
            trajectory.append(state)
        trajectories.append(np.array(trajectory))
    return trajectories

In [ ]:
# Generate dataset
trajectories = generate_ball_dataset(n_balls, dt, seq_len + pred_len, box_size)

In [ ]:
# Dataset Class
class BallTrajectoryDataset(Dataset):
    def __init__(self, trajectories, seq_len, pred_len):
        self.trajectories = trajectories
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        return len(self.trajectories)

    def __getitem__(self, idx):
        trajectory = self.trajectories[idx]
        input_seq = trajectory[:self.seq_len]
        target_seq = trajectory[self.seq_len:self.seq_len + self.pred_len]
        return torch.tensor(input_seq, dtype=torch.float32), torch.tensor(target_seq, dtype=torch.float32)

In [ ]:
# Save Trajectory GIFs
def save_trajectory_gif(trajectory, radius, box_size, save_path, title="Trajectory"):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(-box_size, box_size)
    ax.set_ylim(-box_size, box_size)
    ax.set_aspect('equal')
    ax.set_title(title)

    ball = plt.Circle((trajectory[0, 0], trajectory[0, 2]), radius, fc='blue')
    ax.add_patch(ball)

    def update(frame):
        ball.set_center((trajectory[frame, 0], trajectory[frame, 2]))
        return ball,

    ani = FuncAnimation(fig, update, frames=len(trajectory), blit=True, interval=50)
    ani.save(save_path, fps=20, writer='imagemagick')
    plt.close(fig)

In [ ]:
# Save Best and Worst Predictions
def save_best_worst_predictions(predictions, targets, losses, folder, radius, box_size, mode):
    os.makedirs(folder, exist_ok=True)
    real_folder = os.path.join(folder, "real_movement")
    predicted_folder = os.path.join(folder, "predicted_movement")
    os.makedirs(real_folder, exist_ok=True)
    os.makedirs(predicted_folder, exist_ok=True)

    sorted_indices = np.argsort(losses)
    best_indices = sorted_indices[:5]
    worst_indices = sorted_indices[-5:]

    for idx in best_indices:
        save_trajectory_gif(targets[idx][-len(targets[idx]) // 5:], radius, box_size, os.path.join(real_folder, f"{mode}_best_real_{idx}.gif"), "True Trajectory (Best)")
        save_trajectory_gif(predictions[idx][-len(predictions[idx]) // 5:], radius, box_size, os.path.join(predicted_folder, f"{mode}_best_predicted_{idx}.gif"), "Predicted Trajectory (Best)")

    for idx in worst_indices:
        save_trajectory_gif(targets[idx][-len(targets[idx]) // 5:], radius, box_size, os.path.join(real_folder, f"{mode}_worst_real_{idx}.gif"), "True Trajectory (Worst)")
        save_trajectory_gif(predictions[idx][-len(predictions[idx]) // 5:], radius, box_size, os.path.join(predicted_folder, f"{mode}_worst_predicted_{idx}.gif"), "Predicted Trajectory (Worst)")

In [ ]:
# Save All Test Predictions
def save_all_test_predictions(predictions, targets, folder, radius, box_size):
    os.makedirs(folder, exist_ok=True)
    real_folder = os.path.join(folder, "real_movement")
    predicted_folder = os.path.join(folder, "predicted_movement")
    os.makedirs(real_folder, exist_ok=True)
    os.makedirs(predicted_folder, exist_ok=True)

    for idx in range(len(predictions)):
        save_trajectory_gif(targets[idx][-len(targets[idx]) // 5:], radius, box_size, os.path.join(real_folder, f"test_real_{idx}.gif"), "True Trajectory")
        save_trajectory_gif(predictions[idx][-len(predictions[idx]) // 5:], radius, box_size, os.path.join(predicted_folder, f"test_predicted_{idx}.gif"), "Predicted Trajectory")

In [ ]:
# Hybrid Model
class SINDyKoopmanFNO(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim, sparse_dim, fourier_modes):
        super(SINDyKoopmanFNO, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim),
        )
        self.koopman = nn.Linear(latent_dim, latent_dim, bias=False)
        self.sindy_dynamics = nn.Linear(latent_dim, sparse_dim, bias=False)
        self.fourier_layer = nn.Sequential(
            nn.Linear(latent_dim, fourier_modes),
            nn.ReLU(),
            nn.Linear(fourier_modes, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
        )

    def forward(self, x):
        z = self.encoder(x)
        z_next = self.koopman(z)
        dzdt = self.sindy_dynamics(z)
        z_fourier = self.fourier_layer(z)
        z_combined = z_next + z_fourier
        x_reconstructed = self.decoder(z_combined)
        return x_reconstructed, z_next, dzdt, z

In [ ]:
# Loss Functions
def compute_energy(state, mass, g):
    vx, vy = state[:, :, 1], state[:, :, 3]
    y = state[:, :, 2]
    kinetic_energy = 0.5 * mass * (vx**2 + vy**2)
    potential_energy = mass * g * y
    return kinetic_energy + potential_energy

def energy_loss(predicted_state, true_state, mass, g):
    predicted_energy = compute_energy(predicted_state, mass, g)
    true_energy = compute_energy(true_state, mass, g)
    return torch.mean((predicted_energy - true_energy)**2)

def combined_loss(x_reconstructed, x_target, z_next, dzdt, z_pred, mass, g, alpha=0.1, beta=0.1, gamma=0.1):
    reconstruction_loss = torch.mean((x_reconstructed - x_target)**2)
    koopman_loss = torch.mean((z_next - z_pred)**2)
    sparse_loss = torch.mean((dzdt - z_pred)**2)
    energy_loss_value = energy_loss(x_reconstructed, x_target, mass, g)
    return reconstruction_loss + alpha * energy_loss_value + beta * koopman_loss + gamma * sparse_loss

In [ ]:
# Dataset Splitting
def split_dataset(trajectories, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1."
    train_val_data, test_data = train_test_split(trajectories, test_size=test_ratio, random_state=42)
    train_data, val_data = train_test_split(train_val_data, test_size=val_ratio / (train_ratio + val_ratio), random_state=42)
    return train_data, val_data, test_data

In [ ]:
train_set, val_set, test_set = split_dataset(trajectories)
train_dataset = BallTrajectoryDataset(train_set, seq_len, pred_len)
val_dataset = BallTrajectoryDataset(val_set, seq_len, pred_len)
test_dataset = BallTrajectoryDataset(test_set, seq_len, pred_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Training and Validation
def train_and_save_predictions(model, train_loader, val_loader, optimizer, mass, g, num_epochs, alpha, beta, gamma, radius, box_size):
    model.train()
    for epoch in range(num_epochs):
        total_train_loss, total_train_acc = 0.0, 0.0
        total_val_loss, total_val_acc = 0.0, 0.0
        train_predictions, train_targets, train_losses = [], [], []

        # Training loop
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            x_reconstructed, z_next, dzdt, z = model(inputs)
            z_pred = model.koopman(z)
            loss = combined_loss(x_reconstructed, targets, z_next, dzdt, z_pred, mass, g, alpha, beta, gamma)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            train_acc = 1 - (torch.norm(x_reconstructed - targets) / torch.norm(targets))
            total_train_acc += train_acc.item()

            train_predictions.append(x_reconstructed.detach().cpu().numpy())
            train_targets.append(targets.detach().cpu().numpy())
            train_losses.append(loss.item())

        val_predictions, val_targets, val_losses = [], [], []
        model.eval()
        with torch.no_grad():
            for inputs, targets in val_loader:
                x_reconstructed, z_next, dzdt, z = model(inputs)
                z_pred = model.koopman(z)
                loss = combined_loss(x_reconstructed, targets, z_next, dzdt, z_pred, mass, g, alpha, beta, gamma)
                total_val_loss += loss.item()
                val_acc = 1 - (torch.norm(x_reconstructed - targets) / torch.norm(targets))
                total_val_acc += val_acc.item()

                val_predictions.append(x_reconstructed.detach().cpu().numpy())
                val_targets.append(targets.detach().cpu().numpy())
                val_losses.append(loss.item())

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {total_train_loss / len(train_loader):.4f}, Train Accuracy: {total_train_acc / len(train_loader):.4f}, "
              f"Val Loss: {total_val_loss / len(val_loader):.4f}, Val Accuracy: {total_val_acc / len(val_loader):.4f}")

        if epoch == num_epochs - 1:
            save_best_worst_predictions(np.concatenate(train_predictions), np.concatenate(train_targets), train_losses, "training_predictions", radius, box_size, "train")
            save_best_worst_predictions(np.concatenate(val_predictions), np.concatenate(val_targets), val_losses, "val_predictions", radius, box_size, "val")

In [ ]:
# Initialize model and optimizer
model = SINDyKoopmanFNO(input_dim=4, latent_dim=latent_dim, hidden_dim=hidden_dim, sparse_dim=sparse_dim, fourier_modes=fourier_modes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
train_and_save_predictions(model, train_loader, val_loader, optimizer, mass, G, num_epochs, alpha, beta, gamma, radius, box_size)

In [ ]:
# Test Evaluation
def evaluate_and_save_test_predictions(model, test_loader, mass, g, radius, box_size):
    model.eval()
    test_predictions, test_targets = [], []
    total_test_loss, total_test_acc = 0.0, 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            x_reconstructed, z_next, dzdt, z = model(inputs)
            loss = combined_loss(x_reconstructed, targets, z_next, dzdt, z, mass, g)
            total_test_loss += loss.item()
            test_acc = 1 - (torch.norm(x_reconstructed - targets) / torch.norm(targets))
            total_test_acc += test_acc.item()

            test_predictions.append(x_reconstructed.detach().cpu().numpy())
            test_targets.append(targets.detach().cpu().numpy())

    avg_test_loss = total_test_loss / len(test_loader)
    avg_test_acc = total_test_acc / len(test_loader)
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {avg_test_acc:.4f}")

    save_all_test_predictions(np.concatenate(test_predictions), np.concatenate(test_targets), folder="test_predictions", radius=radius, box_size=box_size)

In [ ]:
# Evaluate the model
evaluate_and_save_test_predictions(model, test_loader, mass, G, radius, box_size)